In [ ]:
!pip install feature_engine

In [ ]:
#Autocomplete
%config Completer.use_jedi = False

import pandas as pd
import numpy as np, numpy.random
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import minmax_scale
from pandas_profiling import ProfileReport
from sklearn.model_selection import KFold

import xgboost as xgb
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor

from sklearn import model_selection

from pathlib import Path
import os, copy
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from pylab import rcParams        
rcParams['figure.figsize'] = 15,15

input_path = Path('/kaggle/input/tabular-playground-series-aug-2021/')

RANDOM_SEED = 42

# Import data

In [ ]:
train = pd.read_csv('/kaggle/input/train-sets/train_set.csv')
validation = pd.read_csv('/kaggle/input/train-sets/validation_set.csv')
y_test = validation['loss']

In [ ]:
cb_submission = pd.read_csv('/kaggle/input/submission-sets/cb_submission.csv')
lgbm_submission = pd.read_csv('/kaggle/input/submission-sets/lgbm_submission.csv')
xgb_submission = pd.read_csv('/kaggle/input/submission-sets/xgb_submission.csv')

# Looking for weights (randomly)

In [ ]:
num = 1000

submission = np.zeros([num, 4])
submission = pd.DataFrame(submission)
submission.columns = ['weight_cb', 'weight_lgbm', 'weight_xgb', 'rmse']

for i in range(num):
    weight = np.random.dirichlet(np.ones(4), size=1)
    submission['weight_cb'][i] = weight[0][0]
    submission['weight_lgbm'][i] = weight[0][1]
    submission['weight_xgb'][i] = weight[0][2]
    y_pred = cb_submission.loss * submission['weight_cb'][i] + lgbm_submission.loss * submission['weight_lgbm'][i] + xgb_submission.loss * submission['weight_xgb'][i]
    submission['rmse'][i] = mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
submission.sort_values('rmse').to_csv("/kaggle/working/weights.csv", index=False)